In [30]:
##Optional, could also use num2str or something else. 
!pip install inflect 

Defaulting to user installation because normal site-packages is not writeable


In [57]:
#Import Required Libraries and Packages
import numpy as np
import pandas as pd 
import inflect as inf
from re import sub
import requests
import bs4

In [34]:
##########################################...HELPER FUNCTIONS...####################################################

In [39]:
###CAMELCASE FUNCTION###
def camel_case(s):
    #not true camelcase but adjusted for queries 
    
  s = sub(r"(_|-)+", " ", s).title().replace(" ", "")
  return ''.join([s[0].upper(), s[1:]])


In [40]:
oscars = pd.read_csv("BigML_Dataset_5f50a62c2fb31c516d000176.csv")
oscars['genre'] = [str(i).split("|") for i in oscars["genre"]]
[i for i in oscars.columns]


['year',
 'movie',
 'movie_id',
 'certificate',
 'duration',
 'genre',
 'rate',
 'metascore',
 'synopsis',
 'votes',
 'gross',
 'release_date',
 'user_reviews',
 'critic_reviews',
 'popularity',
 'awards_wins',
 'awards_nominations',
 'Oscar_Best_Picture_won',
 'Oscar_Best_Picture_nominated',
 'Oscar_Best_Director_won',
 'Oscar_Best_Director_nominated',
 'Oscar_Best_Actor_won',
 'Oscar_Best_Actor_nominated',
 'Oscar_Best_Actress_won',
 'Oscar_Best_Actress_nominated',
 'Oscar_Best_Supporting_Actor_won',
 'Oscar_Best_Supporting_Actor_nominated',
 'Oscar_Best_Supporting_Actress_won',
 'Oscar_Best_Supporting_Actress_nominated',
 'Oscar_Best_AdaScreen_won',
 'Oscar_Best_AdaScreen_nominated',
 'Oscar_Best_OriScreen_won',
 'Oscar_Best_OriScreen_nominated',
 'Oscar_nominated',
 'Oscar_nominated_categories',
 'Golden_Globes_won',
 'Golden_Globes_won_categories',
 'Golden_Globes_nominated',
 'Golden_Globes_nominated_categories',
 'BAFTA_won',
 'BAFTA_won_categories',
 'BAFTA_nominated',
 'BAFTA_

In [41]:
oscar = oscars[["movie", 'year','release_date.year', 'Oscar_nominated', 'Oscar_nominated_categories', 'Oscar_Best_Picture_won' ,'gross' ,'genre','rate','metascore']]
oscar.sort_values(['year', 'movie'])

,movie,year,release_date.year,Oscar_nominated,Oscar_nominated_categories,Oscar_Best_Picture_won,gross,genre,rate,metascore
94,102 Dalmatians,2000,2000.0,1,Best Costume Design,No,66940000.0,"[Adventure, Comedy, Family]",4.8,35.0
51,Almost Famous,2000,2000.0,4,Best Original Screenplay|Best Performance by a...,No,32520000.0,"[Adventure, Comedy, Drama]",7.9,90.0
18,American Psycho,2000,2000.0,0,NaN,No,15050000.0,"[Crime, Drama]",7.6,64.0
134,Amores Perros,2000,2001.0,1,Best Foreign Language Film,No,5380000.0,"[Drama, Thriller]",8.1,83.0
57,Battlefield Earth,2000,2000.0,0,NaN,No,21470000.0,"[Action, Adventure, Sci-Fi]",2.4,9.0
...,...,...,...,...,...,...,...,...,...,...
1176,The Shape of Water,2017,2017.0,13,Best Achievement in Directing|Best Motion Pict...,No,37900000.0,"[Adventure, Drama, Fantasy]",7.9,86.0
1169,"Three Billboards Outside Ebbing, Missouri",2017,NaN,7,Best Motion Picture of the Year|Best Performan...,No,37250000.0,"[Crime, Drama]",8.3,88.0
1180,Victoria & Abdul,2017,2017.0,2,Best Achievement in Makeup and Hairstyling|Bes...,No,22250000.0,"[Biography, Drama, History]",6.8,58.0
1118,War for the Planet of the Apes,2017,2017.0,1,Best Achievement in Visual Effects,No,146880000.0,"[Action, Adventure, Drama]",7.5,82.0


In [42]:
##OSCAR DATA, we need to keep title, some numeric ratings, year, genre, and whether or not it was simply nominated 
# or it won

##Then We need a way to scrape movie tropes and keep them organized in a list 

In [91]:
##Scrape Data from TV tropes--
# /Film for the film. 
# Place Animated movies in different places




#first, it pulls from oscar the movietitle. 

titles = [oscar["movie"][i] for i in range(0, len(list(oscar["movie"])))]

#if there's numbers in the title then they must be convertyed into text. We can write a small function to do that
# or use a library like num2str or inflect
p = inf.engine()
#then, we read the web, and scrape the data that we need from the website
#this will be the t



## Ex. Star Wars : Ep II -- Arr.... only the Attack of the Clones Part is important.
# & must be converted too and other delimeters must be cleaned

filmtitle = camel_case(titles[3])

urlFilm = f'https://tvtropes.org/pmwiki/pmwiki.php/Film/{filmtitle}'
urlAnim = f'https://tvtropes.org/pmwiki/pmwiki.php/WesternAnimation/'


#GET PAGE 
page = requests.get(urlFilm)



#BS the rquest 
soup = bs4.BeautifulSoup(page.content, 'html.parser')

print(soup.find_all('a', attrs = {'class' : 'twikilink'}))

#use regex to filter out the strings that have an href that goes to anything off the mainwiki
#-- that way we can get only tropes



#3,6. 






[<a class="twikilink" href="/pmwiki/pmwiki.php/Creator/FridaKahlo" title="/pmwiki/pmwiki.php/Creator/FridaKahlo">Frida Kahlo</a>, <a class="twikilink" href="/pmwiki/pmwiki.php/Creator/SalmaHayek" title="/pmwiki/pmwiki.php/Creator/SalmaHayek">Salma Hayek</a>, <a class="twikilink" href="/pmwiki/pmwiki.php/UsefulNotes/AcademyAward" title="/pmwiki/pmwiki.php/UsefulNotes/AcademyAward">Academy Award</a>, <a class="twikilink" href="/pmwiki/pmwiki.php/Creator/AlfredMolina" title="/pmwiki/pmwiki.php/Creator/AlfredMolina">Alfred Molina</a>, <a class="twikilink" href="/pmwiki/pmwiki.php/Creator/GeoffreyRush" title="/pmwiki/pmwiki.php/Creator/GeoffreyRush">Geoffrey Rush</a>, <a class="twikilink" href="/pmwiki/pmwiki.php/UsefulNotes/LeonTrotsky" title="/pmwiki/pmwiki.php/UsefulNotes/LeonTrotsky">Leon Trotsky</a>, <a class="twikilink" href="/pmwiki/pmwiki.php/Creator/EdwardNorton" title="/pmwiki/pmwiki.php/Creator/EdwardNorton">Edward Norton</a>, <a class="twikilink" href="/pmwiki/pmwiki.php/Main/Ad

In [ ]:
# /Grab all the tropes

